# Chapter 03-01: 손실 함수 (Loss Functions)

## 학습 목표
- 회귀와 분류 문제에 적합한 손실 함수를 이해하고 선택할 수 있다
- `from_logits=True`와 `False`의 차이를 이해하고 올바르게 사용할 수 있다
- 커스텀 손실 함수를 함수형과 클래스형으로 작성할 수 있다

## 목차
1. [수학적 기초](#1.-수학적-기초)
2. [회귀 손실 함수: MSE, MAE, Huber](#2.-회귀-손실-함수)
3. [이진 분류: Binary Cross-Entropy](#3.-이진-분류-손실-함수)
4. [다중 분류: Categorical vs Sparse Categorical](#4.-다중-분류-손실-함수)
5. [커스텀 손실 함수](#5.-커스텀-손실-함수)
6. [정리](#6.-정리)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow 버전:", tf.__version__)

# 재현성을 위한 시드 고정
tf.random.set_seed(42)
np.random.seed(42)

## 1. 수학적 기초

### 회귀 손실 함수

**평균 제곱 오차 (MSE, Mean Squared Error)**

$$L_{MSE} = \frac{1}{N}\sum_{i=1}^N (\hat{y}_i - y_i)^2$$

- 이상치(outlier)에 매우 민감 (오차를 제곱하기 때문)
- 미분 가능하여 최적화에 유리

**평균 절대 오차 (MAE, Mean Absolute Error)**

$$L_{MAE} = \frac{1}{N}\sum_{i=1}^N |y_i - \hat{y}_i|$$

- 이상치에 덜 민감
- $x=0$ 에서 미분 불가능

**Huber Loss**

$$L_\delta = \begin{cases} \frac{1}{2}x^2 & \text{if } |x| \leq \delta \\ \delta|x| - \frac{1}{2}\delta^2 & \text{otherwise} \end{cases}$$

- MSE와 MAE의 장점을 결합
- $\delta$ 범위 내에서 MSE처럼, 밖에서 MAE처럼 동작

### 분류 손실 함수

**이진 크로스 엔트로피 (BCE, Binary Cross-Entropy)**

$$L_{BCE} = -[y\log p + (1-y)\log(1-p)]$$

**범주형 크로스 엔트로피 (CCE, Categorical Cross-Entropy)**

$$L_{CCE} = -\sum_i y_i \log p_i$$

---
### 🐣 초등학생을 위한 손실 함수 친절 설명!

#### 🤔 손실 함수(Loss Function)가 뭐예요?

손실 함수는 **'AI가 얼마나 틀렸는지 점수를 매기는 도구'**예요!

> 💡 **비유**: 시험 채점 선생님처럼!
> - 정답: 고양이
> - AI 예측: 고양이(확신 90%) → 손실 낮음 😊
> - AI 예측: 강아지(확신 80%) → 손실 높음 😱

학습이란 이 손실 점수를 **최대한 낮추는 과정**이에요!

---

#### 📏 회귀 손실 함수: 숫자를 예측할 때

**MSE (평균 제곱 오차)**:
- 틀린 양을 제곱하여 더해요
- 크게 틀릴수록 **훨씬** 더 큰 벌점!

```
오차: 1 → 벌점 1²  = 1
오차: 5 → 벌점 5²  = 25   ← 5배 틀렸는데 25배 벌점!
오차: 10 → 벌점 10² = 100
```

> 💡 이상치(크게 틀린 데이터)에 엄청 민감해요!

**MAE (평균 절대 오차)**:
- 틀린 양을 그냥 더해요 (절대값)
- 이상치에 덜 민감하지만, 경사(기울기)가 불연속적이에요

**Huber Loss**:
- 작은 오차 → MSE처럼 (부드럽고 미분 가능)
- 큰 오차 → MAE처럼 (이상치에 강건)
- **두 장점을 합친 '중간 다리' 역할!**

---

#### 🏷️ 이진 크로스 엔트로피 (Binary Cross-Entropy, BCE)

**언제 쓰나요?** 답이 **둘 중 하나**인 문제!
- 스팸 메일 vs 정상 메일 📧
- 고양이 사진 vs 고양이 아닌 사진 🐱
- 병이 있는지 vs 없는지 🏥

**어떻게 작동하나요?**

$$L_{BCE} = -[y \cdot \log p + (1-y) \cdot \log(1-p)]$$

| 정답 | AI 예측 | 손실 |
|------|---------|------|
| 고양이(y=1) | 고양이 확률 0.9 | 낮음 😊 (-log(0.9) ≈ 0.1) |
| 고양이(y=1) | 고양이 확률 0.1 | 매우 높음 😱 (-log(0.1) ≈ 2.3) |
| 아님(y=0)   | 고양이 확률 0.05 | 낮음 😊 |

> 💡 **핵심 특성**: 자신 있게 틀릴수록 훨씬 큰 벌점!
> log 함수가 0에 가까울수록 무한대로 커지기 때문이에요.

**from_logits=True vs False의 차이**:
- `False`: 모델이 이미 Sigmoid를 거쳐서 0~1 확률을 출력
- `True`: 모델이 Sigmoid 없이 raw 값 출력 → 내부에서 Sigmoid 적용
- ⭐ `True`가 더 안정적! (log(0) 문제를 내부에서 수치적으로 처리)

---

#### 🏷️ 범주형 크로스 엔트로피 (Categorical Cross-Entropy, CCE)

**언제 쓰나요?** 답이 **여러 개 중 하나**인 문제!
- 손글씨 숫자 0~9 중 어느 것? 🔢
- 이미지가 개/고양이/토끼/새 중 어느 것? 🐾
- 뉴스가 정치/경제/스포츠/문화 중 어느 분야? 📰

$$L_{CCE} = -\sum_i y_i \log p_i$$

정답 클래스에 해당하는 확률($p_i$)만 손실에 영향을 줘요!

**Categorical vs Sparse Categorical의 차이**:

```
Categorical:  y = [0, 1, 0, 0]  ← One-hot 형식 (3개 중 1번 클래스)
Sparse:       y = 1              ← 정수 인덱스 형식 (1번 클래스)
```

수학적으로 완전히 동일한 결과! 레이블 형식에 따라 선택해요.

> 💡 **실무 팁**: 보통 정수 레이블이 더 편리해서
> `SparseCategoricalCrossentropy`를 더 많이 써요!

## 2. 회귀 손실 함수

MSE, MAE, Huber Loss를 비교해본다. 특히 **이상치(outlier)**가 있을 때의 차이에 주목하자.

In [ ]:
# ---------------------------------------------------
# 회귀 손실 함수 비교: MSE, MAE, Huber
# ---------------------------------------------------

# TensorFlow 손실 함수 객체 생성
mse_loss = tf.keras.losses.MeanSquaredError()
mae_loss = tf.keras.losses.MeanAbsoluteError()
huber_loss = tf.keras.losses.Huber(delta=1.0)  # delta=1.0 기본값

# 이상치 없는 정상 데이터
y_true_normal = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0])
y_pred_normal = tf.constant([1.1, 2.1, 2.9, 4.2, 4.8])

# 이상치가 포함된 데이터 (마지막 값이 이상치)
y_true_outlier = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0])
y_pred_outlier = tf.constant([1.1, 2.1, 2.9, 4.2, 15.0])  # 15.0: 큰 이상치

print("=== 정상 데이터 ===")
print(f"MSE:   {mse_loss(y_true_normal, y_pred_normal).numpy():.4f}")
print(f"MAE:   {mae_loss(y_true_normal, y_pred_normal).numpy():.4f}")
print(f"Huber: {huber_loss(y_true_normal, y_pred_normal).numpy():.4f}")

print("\n=== 이상치 포함 데이터 ===")
print(f"MSE:   {mse_loss(y_true_outlier, y_pred_outlier).numpy():.4f}  <- 이상치에 크게 영향받음")
print(f"MAE:   {mae_loss(y_true_outlier, y_pred_outlier).numpy():.4f}")
print(f"Huber: {huber_loss(y_true_outlier, y_pred_outlier).numpy():.4f}  <- 이상치에 강건")

In [ ]:
# ---------------------------------------------------
# 손실 함수 곡선 시각화
# x축: 예측 오차 (y_pred - y_true)
# y축: 각 손실 함수 값
# ---------------------------------------------------

x = np.linspace(-4, 4, 200)  # 오차 범위
delta = 1.0

# 각 손실 함수 계산
mse_vals = x ** 2
mae_vals = np.abs(x)
huber_vals = np.where(
    np.abs(x) <= delta,
    0.5 * x ** 2,
    delta * np.abs(x) - 0.5 * delta ** 2
)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 왼쪽: 손실 함수 곡선 비교
ax1 = axes[0]
ax1.plot(x, mse_vals, label='MSE', color='blue', linewidth=2)
ax1.plot(x, mae_vals, label='MAE', color='orange', linewidth=2)
ax1.plot(x, huber_vals, label=f'Huber (δ={delta})', color='green', linewidth=2, linestyle='--')
ax1.axvline(x=delta, color='gray', linestyle=':', alpha=0.7, label=f'x=±δ={delta}')
ax1.axvline(x=-delta, color='gray', linestyle=':', alpha=0.7)
ax1.set_xlim(-4, 4)
ax1.set_ylim(0, 10)
ax1.set_xlabel('예측 오차 (y_pred - y_true)')
ax1.set_ylabel('손실값')
ax1.set_title('회귀 손실 함수 비교')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 오른쪽: 이상치 영향 비교 (막대 그래프)
ax2 = axes[1]
categories = ['정상 데이터', '이상치 포함']
mse_vals_bar = [
    mse_loss(y_true_normal, y_pred_normal).numpy(),
    mse_loss(y_true_outlier, y_pred_outlier).numpy()
]
mae_vals_bar = [
    mae_loss(y_true_normal, y_pred_normal).numpy(),
    mae_loss(y_true_outlier, y_pred_outlier).numpy()
]
huber_vals_bar = [
    huber_loss(y_true_normal, y_pred_normal).numpy(),
    huber_loss(y_true_outlier, y_pred_outlier).numpy()
]

x_bar = np.arange(len(categories))
width = 0.25
ax2.bar(x_bar - width, mse_vals_bar, width, label='MSE', color='blue', alpha=0.7)
ax2.bar(x_bar, mae_vals_bar, width, label='MAE', color='orange', alpha=0.7)
ax2.bar(x_bar + width, huber_vals_bar, width, label='Huber', color='green', alpha=0.7)
ax2.set_xticks(x_bar)
ax2.set_xticklabels(categories)
ax2.set_ylabel('손실값')
ax2.set_title('이상치 존재 시 손실 함수 영향 비교')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()
print("MSE는 이상치에 의해 손실값이 크게 증가하는 반면, Huber Loss는 상대적으로 안정적이다.")

## 3. 이진 분류 손실 함수

### Binary Cross-Entropy와 `from_logits` 매개변수

| 설정 | 모델 출력 | 내부 처리 |
|------|-----------|----------|
| `from_logits=False` (기본) | Sigmoid 활성화 후 확률값 (0~1) | 그대로 사용 |
| `from_logits=True` | Sigmoid 적용 전 raw 값 (임의 범위) | 내부에서 Sigmoid 적용 후 계산 |

> **권장**: `from_logits=True`를 사용하면 수치적으로 더 안정적이다 (log(0) 문제 회피).

In [ ]:
# ---------------------------------------------------
# Binary Cross-Entropy: from_logits 비교
# ---------------------------------------------------

# 실제 레이블 (이진: 0 또는 1)
y_true_binary = tf.constant([1.0, 0.0, 1.0, 1.0, 0.0])

# 방법 1: 모델 출력층에 Sigmoid 사용 -> from_logits=False
logits = tf.constant([2.0, -1.0, 3.0, 0.5, -2.0])  # raw 출력
probs = tf.sigmoid(logits)  # Sigmoid 적용 후 확률값

bce_from_probs = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce_from_logits = tf.keras.losses.BinaryCrossentropy(from_logits=True)

loss_from_probs = bce_from_probs(y_true_binary, probs)
loss_from_logits = bce_from_logits(y_true_binary, logits)

print("logits (raw 출력):", logits.numpy())
print("probs  (Sigmoid 후):", probs.numpy().round(4))
print()
print(f"from_logits=False (확률 입력): {loss_from_probs.numpy():.6f}")
print(f"from_logits=True  (로짓 입력): {loss_from_logits.numpy():.6f}")
print("두 값은 수학적으로 동일하지만, from_logits=True가 수치적으로 더 안정적이다")

print("\n=== 모델 정의 예시 ===")
print("""
# 방법 1: from_logits=True (권장)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1)  # 활성화 함수 없음 (logits 출력)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))

# 방법 2: from_logits=False
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid 적용
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False))
""")

## 4. 다중 분류 손실 함수

### Categorical vs Sparse Categorical Cross-Entropy

| 손실 함수 | 레이블 형식 | 사용 예 |
|-----------|------------|--------|
| `CategoricalCrossentropy` | One-hot 인코딩: `[0, 1, 0]` | 레이블이 이미 one-hot인 경우 |
| `SparseCategoricalCrossentropy` | 정수 인덱스: `1` | 레이블이 클래스 번호인 경우 (일반적) |

In [ ]:
# ---------------------------------------------------
# Categorical vs Sparse Categorical Cross-Entropy
# ---------------------------------------------------

# 3개 클래스 분류 예시
# 모델의 Softmax 출력 확률
y_pred_probs = tf.constant([
    [0.7, 0.2, 0.1],  # 샘플 1: 클래스 0 예측
    [0.1, 0.8, 0.1],  # 샘플 2: 클래스 1 예측
    [0.2, 0.1, 0.7],  # 샘플 3: 클래스 2 예측
])

# One-hot 형식 레이블
y_true_onehot = tf.constant([
    [1, 0, 0],  # 클래스 0
    [0, 1, 0],  # 클래스 1
    [0, 0, 1],  # 클래스 2
], dtype=tf.float32)

# 정수 형식 레이블 (Sparse)
y_true_sparse = tf.constant([0, 1, 2])  # 각각 클래스 인덱스

cce = tf.keras.losses.CategoricalCrossentropy()
scce = tf.keras.losses.SparseCategoricalCrossentropy()

loss_cce = cce(y_true_onehot, y_pred_probs)
loss_scce = scce(y_true_sparse, y_pred_probs)

print("레이블 형식 비교:")
print(f"  One-hot 레이블:\n{y_true_onehot.numpy()}")
print(f"  Sparse 레이블: {y_true_sparse.numpy()}")
print()
print(f"CategoricalCrossentropy   (one-hot 입력): {loss_cce.numpy():.6f}")
print(f"SparseCategoricalCrossentropy (정수 입력): {loss_scce.numpy():.6f}")
print("두 손실 값은 동일하다 (같은 계산을 다른 레이블 형식으로 수행)")

print("\n=== from_logits 옵션도 동일하게 적용 ===")
y_pred_logits = tf.constant([
    [2.0, 0.5, 0.1],
    [0.1, 3.0, 0.2],
    [0.3, 0.1, 2.5],
])
scce_logits = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
print(f"SparseCCE from_logits=True: {scce_logits(y_true_sparse, y_pred_logits).numpy():.6f}")

## 5. 커스텀 손실 함수

TensorFlow에서 커스텀 손실 함수를 작성하는 두 가지 방법:

1. **함수형**: 간단한 경우에 적합, `model.compile(loss=my_loss_fn)`
2. **클래스형**: 매개변수가 필요하거나 상태를 가질 때 적합, `tf.keras.losses.Loss` 상속

In [ ]:
# ---------------------------------------------------
# 커스텀 손실 함수: 함수형
# ---------------------------------------------------

def weighted_mse_loss(y_true, y_pred):
    """가중치가 적용된 MSE: 양수 오차에 더 큰 패널티를 부여"""
    error = y_pred - y_true
    # 양수 오차(과대 예측)에 2배 패널티
    weights = tf.where(error > 0, 2.0, 1.0)
    return tf.reduce_mean(weights * tf.square(error))

# 테스트
y_true_test = tf.constant([1.0, 2.0, 3.0])
y_pred_under = tf.constant([0.5, 1.5, 2.5])  # 과소 예측
y_pred_over  = tf.constant([1.5, 2.5, 3.5])  # 과대 예측 (같은 절대 오차)

print("=== 함수형 커스텀 손실 함수 ===")
print(f"과소 예측 손실: {weighted_mse_loss(y_true_test, y_pred_under).numpy():.4f}")
print(f"과대 예측 손실: {weighted_mse_loss(y_true_test, y_pred_over).numpy():.4f}")
print("과대 예측에 2배 패널티가 적용되어 손실이 더 큰 것을 확인")

print("\n=== 모델에 적용하는 방법 ===")
print("""
model.compile(
    optimizer='adam',
    loss=weighted_mse_loss  # 함수를 직접 전달
)
""")

In [ ]:
# ---------------------------------------------------
# 커스텀 손실 함수: 클래스형 (tf.keras.losses.Loss 상속)
# ---------------------------------------------------

class FocalLoss(tf.keras.losses.Loss):
    """Focal Loss: 불균형 데이터셋에서 어려운 샘플에 집중하는 손실 함수
    
    원래 논문: Lin et al., 'Focal Loss for Dense Object Detection', 2017
    
    수식: FL(p_t) = -alpha_t * (1 - p_t)^gamma * log(p_t)
    - gamma: 포커싱 파라미터 (클수록 어려운 샘플에 더 집중)
    - alpha: 클래스 불균형 보정 가중치
    """
    
    def __init__(self, gamma=2.0, alpha=0.25, name='focal_loss'):
        super().__init__(name=name)
        self.gamma = gamma  # 포커싱 파라미터
        self.alpha = alpha  # 클래스 가중치
    
    def call(self, y_true, y_pred):
        # 수치 안정성을 위해 클리핑
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        
        # 이진 크로스 엔트로피 계산
        bce = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        
        # 확률 p_t 계산
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        
        # alpha 가중치 적용
        alpha_t = y_true * self.alpha + (1 - y_true) * (1 - self.alpha)
        
        # Focal Loss = alpha_t * (1 - p_t)^gamma * bce
        focal_loss = alpha_t * tf.pow(1 - p_t, self.gamma) * bce
        
        return tf.reduce_mean(focal_loss)
    
    def get_config(self):
        """직렬화를 위한 설정 반환 (모델 저장 시 필요)"""
        config = super().get_config()
        config.update({'gamma': self.gamma, 'alpha': self.alpha})
        return config

# 테스트
focal_loss = FocalLoss(gamma=2.0, alpha=0.25)
bce_standard = tf.keras.losses.BinaryCrossentropy()

# 쉬운 샘플 (확신 있는 예측)
y_true_easy = tf.constant([1.0, 0.0, 1.0])
y_pred_easy = tf.constant([0.95, 0.05, 0.9])  # 높은 확신

# 어려운 샘플 (불확실한 예측)
y_true_hard = tf.constant([1.0, 0.0, 1.0])
y_pred_hard = tf.constant([0.55, 0.45, 0.6])  # 낮은 확신

print("=== 클래스형 커스텀 손실 함수 (Focal Loss) ===")
print(f"쉬운 샘플 - BCE:   {bce_standard(y_true_easy, y_pred_easy).numpy():.4f}")
print(f"쉬운 샘플 - Focal: {focal_loss(y_true_easy, y_pred_easy).numpy():.4f}  <- 크게 감소")
print(f"어려운 샘플 - BCE:   {bce_standard(y_true_hard, y_pred_hard).numpy():.4f}")
print(f"어려운 샘플 - Focal: {focal_loss(y_true_hard, y_pred_hard).numpy():.4f}  <- 상대적으로 덜 감소")
print("\nFocal Loss는 쉬운 샘플의 영향을 줄여 어려운 샘플 학습에 집중하게 한다")

print("\n설정 정보:", focal_loss.get_config())

## 6. 정리

### 손실 함수 선택 가이드

| 문제 유형 | 권장 손실 함수 | 비고 |
|-----------|--------------|------|
| 회귀 (이상치 없음) | `MeanSquaredError` | 미분 용이, 최적화 안정 |
| 회귀 (이상치 있음) | `Huber` 또는 `MeanAbsoluteError` | Huber가 두 장점 결합 |
| 이진 분류 | `BinaryCrossentropy(from_logits=True)` | 수치 안정성 높음 |
| 다중 분류 (one-hot) | `CategoricalCrossentropy(from_logits=True)` | |
| 다중 분류 (정수 레이블) | `SparseCategoricalCrossentropy(from_logits=True)` | 일반적으로 권장 |
| 불균형 데이터 | `FocalLoss` (커스텀) | 어려운 샘플에 집중 |

### 핵심 정리
- `from_logits=True`는 모델 출력층에 Sigmoid/Softmax가 없을 때 사용하며, 수치적으로 더 안정적이다
- Huber Loss는 `delta` 파라미터로 MSE와 MAE 사이의 균형을 조절한다
- 커스텀 손실 함수는 함수형(간단)과 클래스형(매개변수 필요) 두 방식으로 작성한다

### 다음 챕터 예고
**Chapter 03-02: 옵티마이저 (Optimizers)** - SGD, Adam, RMSprop 등 다양한 최적화 알고리즘의 동작 원리와 선택 방법을 다룬다.